In [30]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
# Define data directories
# base_data_dir = '/srv/scratch/z5370003/projects/data/groundwater/FEFLOW/coastal/variable_density/'
base_data_dir = '/Users/arpitkapoor/Library/CloudStorage/OneDrive-UNSW/Shared/Projects/01_PhD/05_groundwater/data/FEFLOW/variable_density'  # Uncomment for local testing
raw_data_dir = os.path.join(base_data_dir, 'all')
patch_data_dir = os.path.join(base_data_dir, 'filter_patch')

print(f"Base data directory: {base_data_dir}")
print(f"Raw data directory: {raw_data_dir}")
print(f"Patch filtered data directory: {patch_data_dir}")

Base data directory: /Users/arpitkapoor/Library/CloudStorage/OneDrive-UNSW/Shared/Projects/01_PhD/05_groundwater/data/FEFLOW/variable_density
Raw data directory: /Users/arpitkapoor/Library/CloudStorage/OneDrive-UNSW/Shared/Projects/01_PhD/05_groundwater/data/FEFLOW/variable_density/all
Patch filtered data directory: /Users/arpitkapoor/Library/CloudStorage/OneDrive-UNSW/Shared/Projects/01_PhD/05_groundwater/data/FEFLOW/variable_density/filter_patch


In [29]:
import sys
sys.path.append('../')
from src.data.transform import Normalize
from src.data.patch_dataset import GWPatchDataset
from src.data.batch_sampler import PatchBatchSampler

In [4]:
# Read data
df = pd.read_csv(os.path.join(raw_data_dir, '0000.csv'))

# Calculate mean and std of coordinates
coord_mean = df[['X', 'Y', 'Z']].mean().values
coord_std = df[['X', 'Y', 'Z']].std().values

# Print mean and std of coordinates
print(f"Coordinate mean: {coord_mean}")
print(f"Coordinate std: {coord_std}")

# Create coordinate transform
coord_transform = Normalize(mean=coord_mean, std=coord_std)

del df

Coordinate mean: [ 3.57225665e+05  6.45774324e+06 -9.27782248e+00]
Coordinate std: [569.1699999  566.35797379  15.26565618]


In [5]:
# Read data
df = pd.read_csv(os.path.join(raw_data_dir, '0000.csv'))

# Define output columns
obs_cols = ['mass_concentration', 'head', 'pressure']

# Mean and std of output
obs_mean = df[obs_cols].mean().values
obs_std = df[obs_cols].std().values

# Print mean and std of output
print(f"Output mean: {obs_mean}")
print(f"Output std: {obs_std}")

# Define output transform
obs_transform = Normalize(mean=obs_mean, std=obs_std)

del df

Output mean: [1.77942252e+04 3.95881156e-01 9.48469883e+01]
Output std: [1.55859465e+04 2.13080032e-01 1.51226320e+02]


In [6]:
input_window_size = 5
output_window_size = 5

In [7]:
patch_ds = GWPatchDataset(data_path=patch_data_dir, dataset='train', 
                          coord_transform=coord_transform, 
                          obs_transform=obs_transform,
                          input_window_size=input_window_size,
                          output_window_size=output_window_size,
                          target_col_idx=1)
len(patch_ds)

13180

In [8]:
patch_ds[0]['core_in'].shape

torch.Size([405, 5])

In [10]:
patch_ds[0].keys()

dict_keys(['core_in', 'ghost_in', 'core_out', 'ghost_out', 'patch_id', 'core_coords', 'ghost_coords'])

In [33]:
batch_sampler = PatchBatchSampler(patch_ds, batch_size=32)
patch_dl = DataLoader(patch_ds, batch_sampler=batch_sampler)


In [34]:
for patch_data in patch_dl:
    print(patch_data['core_in'].shape, patch_data['ghost_in'].shape, patch_data['core_out'].shape, patch_data['ghost_out'].shape)
    print(patch_data['core_coords'].shape, patch_data['ghost_coords'].shape)
    break

torch.Size([32, 405, 5]) torch.Size([32, 104, 5]) torch.Size([32, 405, 5]) torch.Size([32, 104, 5])
torch.Size([32, 405, 3]) torch.Size([32, 104, 3])


In [9]:
for patch_data in patch_ds:
    print([(k, v.shape) if k != 'patch_id' else (k, v) for k, v in patch_data.items()])
    input_coords = torch.concat([patch_data['core_coords'], patch_data['ghost_coords']], dim=0)
    input_obs = torch.concat([patch_data['core_in'], patch_data['ghost_in']], dim=1)
    output_obs = torch.concat([patch_data['core_out'], patch_data['ghost_out']], dim=1)
    core_len = int(input_coords.shape[0] * 0.8)
    

[('core_in', torch.Size([5, 955])), ('ghost_in', torch.Size([5, 955])), ('core_out', torch.Size([5, 955])), ('ghost_out', torch.Size([5, 955])), ('patch_id', 1), ('core_coords', torch.Size([405, 3])), ('ghost_coords', torch.Size([104, 3]))]
[('core_in', torch.Size([5, 955])), ('ghost_in', torch.Size([5, 955])), ('core_out', torch.Size([5, 955])), ('ghost_out', torch.Size([5, 955])), ('patch_id', 1), ('core_coords', torch.Size([405, 3])), ('ghost_coords', torch.Size([104, 3]))]
[('core_in', torch.Size([5, 955])), ('ghost_in', torch.Size([5, 955])), ('core_out', torch.Size([5, 955])), ('ghost_out', torch.Size([5, 955])), ('patch_id', 1), ('core_coords', torch.Size([405, 3])), ('ghost_coords', torch.Size([104, 3]))]
[('core_in', torch.Size([5, 955])), ('ghost_in', torch.Size([5, 955])), ('core_out', torch.Size([5, 955])), ('ghost_out', torch.Size([5, 955])), ('patch_id', 1), ('core_coords', torch.Size([405, 3])), ('ghost_coords', torch.Size([104, 3]))]
[('core_in', torch.Size([5, 955])), 

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 5 but got size 4 for tensor number 1 in the list.